# Demonstration of Combined Model
####    <p style='text-align: right;'>  --- A Joint Decision Architecture </p>


<img src="./combined_flowchart.jpg" width=1000 height=1000 />

# Part 1: Import database and Trained models

In [ ]:
import torch
import torch.nn as nn
import mini_library as ml
from torchvision import models
    
# define paths for validationa and predictions
val_file_path = './acse-miniproject/train'
test_file_path = './acse-miniproject/test'


# load models
model_1 = models.wide_resnet50_2()
model_2 = models.inception_v3()
model_3 = models.densenet161()
model_4 = models.wide_resnet101_2()

# get models last layer to modify it
num_ftrs_1 = model_1.fc.in_features
num_ftrs_2 = model_2.fc.in_features
num_ftrs_3 = model_3.classifier.in_features
num_ftrs_4 = model_4.fc.in_features


# modify models last layer
model_1.fc = nn.Linear(num_ftrs_1, 200)

model_2.AuxLogits.fc = nn.Linear(768, 200)
model_2.fc = nn.Linear(2048,200)
model_2.aux_logits = False

model_3.classifier = nn.Linear(num_ftrs_3, 200)
model_4.fc = nn.Linear(num_ftrs_4, 200)


# load the models
model_1.load_state_dict(torch.load('./pth_files/resnet50.pth', 
                                    map_location=torch.device('cpu')))
model_2.load_state_dict(torch.load('./pth_files/GoogleNet_inception_v3.pth', 
                                    map_location=torch.device('cpu')))
model_3.load_state_dict(torch.load('./pth_files/Densenet_128.pth', 
                                    map_location=torch.device('cpu')))
model_4.load_state_dict(torch.load('./pth_files/resnet101.pth', 
                                    map_location=torch.device('cpu')))


# Part 2: Evaluate accuracy and get predictions

In [ ]:


# list of image sizes for input of each model
input_sizes = [224, 299, 128, 144]

# list of models to be considered for classification
models_ = [model_1, model_2, model_3, model_4]

# list of models' validation accuracies
model_accs = [0.8292, 0.753, 0.788, 0.787]

# evaluate
ml.evaluate_combined(models_, model_accs, input_sizes, val_file_path)

# get CSV of predictions
ml.get_prediction_csv_combined(models_, model_accs, input_sizes, test_file_path)
